# SF-DAT-21 | Unit Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [25]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as pl
%matplotlib inline

In [2]:
df_raw = pd.read_csv("../../dataset/admissions.csv")
df = df_raw.dropna()
print(df.head())

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### Question 1. Let's create a frequency table of our variables.

In [3]:
# frequency table for prestige and whether or not someone was admitted

prestige = df['prestige']
print('Prestige Frequency Table:')
prestige.value_counts()

Prestige Frequency Table:


2.0    148
3.0    121
4.0     67
1.0     61
Name: prestige, dtype: int64

In [4]:
admit = df['admit']
print('Admit Frequency Table:')
admit.value_counts()

Admit Frequency Table:


0    271
1    126
Name: admit, dtype: int64

## Part 2. Return of dummy variables

#### Question 2.1. Create class or dummy variables for prestige.

In [5]:
prestige_dummy_df = pd.get_dummies(df['prestige'], prefix = 'Prestige')
prestige_dummy_df

,Prestige_1.0,Prestige_2.0,Prestige_3.0,Prestige_4.0
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0
5,0.0,1.0,0.0,0.0
6,1.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0
8,0.0,0.0,1.0,0.0
9,0.0,1.0,0.0,0.0


#### Question 2.2. When modeling our class variables, how many do we need?

Answer: We need to have n-1 variables, since we will know the value of the final variable by process of elimination.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [6]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(prestige_dummy_df.ix[:, 'Prestige_1.0':])
print(handCalc.head())

   admit    gre   gpa  Prestige_1.0  Prestige_2.0  Prestige_3.0  Prestige_4.0
0      0  380.0  3.61           0.0           0.0           1.0           0.0
1      1  660.0  3.67           0.0           0.0           1.0           0.0
2      1  800.0  4.00           1.0           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           0.0           1.0


In [7]:
# crosstab of Prestige 1.0 and admission decision
pd.crosstab(index = handCalc['admit'], columns = handCalc['Prestige_1.0'])

Prestige_1.0,0.0,1.0
admit,,
0,243,28
1,93,33


In [8]:
# frequency table cutting prestige and whether or not someone was admitted
ft = pd.crosstab(index = [handCalc['Prestige_1.0'], handCalc['Prestige_2.0'], handCalc['Prestige_3.0'], handCalc['Prestige_4.0']],
            columns = handCalc['admit'], rownames = ['P1', 'P2', 'P3', 'P4'], margins = True)
ft

admit              0    1  All
P1  P2  P3  P4                
0.0 0.0 0.0 1.0   55   12   67
        1.0 0.0   93   28  121
    1.0 0.0 0.0   95   53  148
1.0 0.0 0.0 0.0   28   33   61
All              271  126  397

#### Question 3.1. Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college.

In [9]:
# this question is ambiguous because the most prestigious college type is prestige #4. I will assume "#1 ranked" refers to the prestige classification of 1
# rather than the most prestigious classification
# odds ratio = p(admit & p1) / 1 - [p(admit & p1)]
(33/61)

0.5409836065573771

#### Question 3.2. Now calculate the odds of admission if you did not attend a #1 ranked college.

In [10]:
# (126-33) = 93
93/(397-33)

0.2554945054945055

#### Question 3.3. Calculate the odds ratio.

In [11]:
# odds ratio p(admission and not p1) / 1-p(admission and not p1)
(93/(397-33))/(1-(93/(397-33)))

0.3431734317343173

#### Question 3.4. Write this finding in a sentence:

Answer: The fact that an applicant's undegraduate degree is from a university with prestige 1.0 is associated with  a 142% increase in likelihood of admission to grad school at UCLA.

#### Question 3.5. Print the cross tab for prestige_4.

In [12]:
pd.crosstab(index = handCalc['Prestige_4.0'], columns = handCalc['admit'], margins = True)

admit,0,1,All
Prestige_4.0,,,
0.0,216,114,330
1.0,55,12,67
All,271,126,397


#### Question 3.6. Calculate the OR.

In [13]:
# p(admitted & prestige = 4) / (1 - p(admitted & prestige = 4))
(12/67)/(1-(12/67))

0.21818181818181817

#### Question 3.7. Write this finding in a sentence.

Answer: Applicants with undergrad prestige of 4 have a 88% lower chance of admission than those applying from schools with lower prestige levels.

## Part 4. Analysis

In [14]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(prestige_dummy_df.ix[:, 'Prestige_1.0':])
print(data.head())

   admit    gre   gpa  Prestige_1.0  Prestige_2.0  Prestige_3.0  Prestige_4.0
0      0  380.0  3.61           0.0           0.0           1.0           0.0
1      1  660.0  3.67           0.0           0.0           1.0           0.0
2      1  800.0  4.00           1.0           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           0.0           1.0


We're going to add a constant term for our Logistic Regression.  The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [15]:
# manually add the intercept
data['intercept'] = 1.0

#### Question 4.1. Set the covariates to a variable called train_cols.

In [16]:
train_cols = data[['gre', 'gpa', 'Prestige_1.0', 'Prestige_2.0', 'Prestige_3.0']]

#### Question 4.2. Fit the model.

In [17]:
logit = sm.Logit(data['admit'], train_cols)
logit_result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.605465
         Iterations 5


#### Question 4.3. Print the summary results.

In [18]:
logit_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      392
Method:                           MLE   Df Model:                            4
Date:                Wed, 08 Jun 2016   Pseudo R-squ.:                 0.03107
Time:                        13:31:52   Log-Likelihood:                -240.37
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                  0.003913
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0011      0.001      1.043      0.297        -0.001     0.003
gpa             -0.5162      0.192     -2.682      0.007        -0.893    -0.139
Prestige_1.0     1.2948      0.385      3.361      0.001         0.540     2.050
Prestige_2.0     0.5120      0.327      1.567      0.117        -0.129     1.153
Prestige_3.0    -0.0504      0.354     -0.142      0.887        -0.745     0.644
================================================================================
"""

#### Question 4.4. Calculate the odds ratios of the coefficients and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params

        conf.columns = ['2.5%', '97.5%', 'OR']
         
hint 3: enumerate what the coefficients mean within the regression equation and solve for their ORs

admit = 1.0 + 0.0011*GRE + -0.5162*GPA + 1.2948*P1 + 0.5120*P2 + -0.0504*P3

In [19]:
# calculate the odds ratio of the coefficients of each IV
# cancelling out the log by taking the exponent
np.exp(logit_result.params)

gre             1.001069
gpa             0.596794
Prestige_1.0    3.650274
Prestige_2.0    1.668639
Prestige_3.0    0.950821
dtype: float64

In [37]:
# use the coefficients / odds ratios to find the 95% CIs for each IV
# lower endpoint: coef - 1.96*(STDV/sqrt(n))
# upper: coef + 1.96*(STDV/sqrt(n))

# gre lower
gre_cil = 0.0011 - 1.96*(0.001/math.sqrt(397))

# gre upper
gre_ciu = 0.0011 + 1.96*(0.001/math.sqrt(397))

gre_ci = (gre_cil, gre_ciu)
gre_ci

(0.001001630419807215, 0.0011983695801927852)

In [39]:
# gpa lower
gpa_cil = -0.5162 - 1.96*(0.192/math.sqrt(397))

# gpa upper
gpa_ciu = -0.5162 + 1.96*(0.192/math.sqrt(397))

gpa_ci = (gpa_cil, gpa_ciu)
gpa_ci

(-0.5350869593970148, -0.49731304060298526)

In [40]:
# p1 lower
p1_cil = 1.2948 - 1.96*(0.385/math.sqrt(397))

# p1 upper
p1_ciu = 1.2948 + 1.96*(0.385/math.sqrt(397))

p1_ci = (p1_cil, p1_ciu)
p1_ci

(1.2569277116257778, 1.3326722883742221)

In [41]:
# p2 lower
p2_cil = 0.5120 - 1.96*(0.327/math.sqrt(397))

# p2 upper
p2_ciu = 0.5120 + 1.96*(0.327/math.sqrt(397))

p2_ci = (p2_cil, p2_ciu)
p2_ci

(0.47983314727695925, 0.5441668527230408)

In [42]:
# p3 lower
p3_cil = -0.0504 - 1.96*(0.354/math.sqrt(397))

# p3 upper
p3_ciu = -0.0504 + 1.96*(0.354/math.sqrt(397))

p3_ci = (p3_cil, p3_ciu)
p3_ci

(-0.08522283138824593, -0.015577168611754066)

#### Question 4.5. Interpret the OR of Prestige_2.

Answer: Applying from a school with prestige of two is associated with a 66% increase in likelihood of admission compared to applying from schools with prestige != 2.

#### Question 4.6. Interpret the OR of GPA.

Note: GPA is a continuous variable, so the odds ratio is a density function (finding the area under a subset of a curve) rather than a simple ratio

Answer: A one unit increase in GPA corresponds to a 0.4 unit decrease in probability of admission.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values.  This will allow us to see how the predicted probability of admission increases/decreases across different variables.  First, we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa".  This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [21]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [22]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)

print(gres)
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])

gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)

print(gpas)
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])

# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [0,1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:42: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


NameError: name 'xrange' is not defined

#### Question 5.1. Recreate the dummy variables.

In [ ]:
# recreate the dummy variables
pd.get_dummies(df['prestige'], prefix=None)

In [ ]:
# keep only what we need for making predictions
???

#### Question 5.2. Make predictions on the enumerated dataset.

In [ ]:
???

#### Question 5.3. Interpret findings for the last 4 observations.

Answer: ???

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.

In [23]:
???

Object `` not found.
